Example Script convertir String en Uppercase


In [ ]:
strings = [
    "PatientNumber",
    "Hospital",
    "StartDateTime",
    "Quantity",
    "ServiceCode",
    "Extra:PrimaryProcedure",
    "EncounterNumber",
    "ServicingDepartment",
    "Duration",
    "ActualCharge",
    "EndDateTime",
    "PointOfService1",
    "Extra:ServiceDescription",
    "Extra:ServiceGroup",
    "Extra:LastUpdateDateTime",
    "Consultant",
    "Extra:ConsultantName",
    "ConsultantSpecialty",
    "Clinic",
    "OrderDateTime",
    "Extra:PriorityCode",
    "Extra:Priority",
    "Extra:StatusCode",
    "Extra:StartDateTreatmentPlan",
    "Extra:EndDateTreatmentPlan",
    "Extra:RequestNo",
    "Extra:OrderingDepartment",
    "Extra:PrivateInsurance",
    "Extra:OriginalServiceCode",
    "Extra:OriginalServiceDesc",
    "Extra:OriginalServiceGroup",
    "Extra:RadiographerExamDuration",
    "Extra:RadiologistLicence",
    "Extra:RadiologistName",
    "Extra:RadiologistSpecialty",
    "Extra:RadiologistReportDateTime",
    "Extra:RadiologistFinalisationDate",
    "Extra:RadiologistReportDuration",
    "Extra:StaffSignoff",
    "Extra:CollectionTime",
    "Extra:SampleReceivedTime",
    "TestResult",
    "Extra:SignatureDateTime",
    "Extra:PathologistName",
    "Extra:PathologistLicence",
    "Extra:ServiceGroupDesc",
    "Extra:DIN",
    "Extra:StartDispenseTime",
    "Extra:PrescriptionValidationTime",
    "Extra:QuantityAdministered",
    "Extra:QuantityPrescribed",
    "Extra:ProcedureSpecialty",
    "Extra:ElectiveOrEmergency",
    "Extra:PreOpStart",
    "Extra:PreOpEnd",
    "Extra:AnaethesiaStart",
    "Extra:AnaethesiaEnd",
    "Extra:RecoveryStart",
    "Extra:RecoveryEnd",
    "Extra:NumberXtraMedicalStaff",
    "Extra:NumberExtraPersons",
    "Extra:NumberTheatreNurses",
    "Extra:NumberTheatreNursesAux",
    "Extra:OncologyFlag",
    "Extra:PatientType",
    "Extra:CancellationDate",
    "Extra:CancellationReasonCode",
    "Extra:CancellationReasonDesc",
    "Extra:AnaesthetistCode",
    "Extra:AnaesthetistName",
    "Extra:AnaestheticTechnique",
    "Extra:RequestStatus",
    "Extra:PlannedSurgeryDate",
    "Extra:OperationID",
    "Extra:OperationStatus",
    "Extra:PACUDuration",
    "Extra:Implants",
    "Extra:Site",
    "Extra:TestName",
    "Extra:OrderingConsultant",
    "Extra:OrderingConsultantSpecialty"
]



converted_strings = []

for string in strings:
    converted_string = string.upper().replace(":", "_")
    converted_strings.append(converted_string)

for string in converted_strings:
    print(string)


Chargement du Dataframe

In [14]:
import pandas as pd

# Chargement du fichier CSV
data = pd.read_csv('../../../../Documents/test.csv', sep=";")

dataWithoutFileType = data.drop(columns="TypeFichier")
display(dataWithoutFileType)
display(len(dataWithoutFileType))

dataWithoutDuplicates = dataWithoutFileType.drop_duplicates()
display(len(dataWithoutDuplicates))

# Initialisation de la requête SQL
query = "INSERT IGNORE INTO FieldMapping (DatabaseField, MappedField) VALUES "




,Nifi Input file,Nifi Output file
0,MRN Number,PATIENTNUMBER
1,Hospital,EXTRA_HOSPITAL
2,DateOfBirth,DATEOFBIRTH
3,Gender,GENDER
4,PatientDeceased,EXTRA_PATIENTDECEASED
...,...,...
217,Implants,EXTRA_IMPLANTS
218,Site,EXTRA_SITE
219,TestName,EXTRA_TESTNAME
220,OrderingConsultant,EXTRA_ORDERINGCONSULTANT


222

194

Création de la QUERY pour FieldMapping

In [15]:
query = "INSERT INTO FieldMapping (DatabaseField, MappedField)\nVALUES\n"

for index, row in dataWithoutDuplicates.iterrows():
    db_field = row[0]
    mapped_field = row[1]
    query += f"    ('{db_field}', '{mapped_field}'),\n"

# Suppression de la dernière virgule et ajout du point-virgule de fin de requête
query = query[:-2] + ';'

# Affichage de la requête
print(query)

INSERT INTO FieldMapping (DatabaseField, MappedField)
VALUES
    ('MRN Number', 'PATIENTNUMBER'),
    ('Hospital', 'EXTRA_HOSPITAL'),
    ('DateOfBirth', 'DATEOFBIRTH'),
    ('Gender', 'GENDER'),
    ('PatientDeceased', 'EXTRA_PATIENTDECEASED'),
    ('DateofDeath', 'EXTRA_DATEOFDEATH'),
    ('PlaceOfBirth', 'EXTRA_PLACEOFBIRTH'),
    ('EthnicOrigin', 'ETHNICORIGIN'),
    ('Nationality', 'EXTRA_NATIONALITY'),
    ('LastName', 'LASTNAME'),
    ('FirstName', 'FIRSTNAME'),
    ('Title', 'TITLE'),
    ('MothersName', 'EXTRA_MOTHERSLASTNAME'),
    ('MothersPreName', 'EXTRA_MOTHERSFIRSTNAME'),
    ('FathersName', 'EXTRA_FATHERSLASTNAME'),
    ('FathersPreName', 'EXTRA_FATHERSFIRSTNAME'),
    ('FamilyDoctor', 'EXTRA_FAMILYDOCTOR'),
    ('BloodRefusal', 'EXTRA_BLOODREFUSAL'),
    ('OrganDonor', 'EXTRA_ORGANDONOR'),
    ('PrefLanguage', 'EXTRA_PREFLANGUAGE'),
    ('LastUpdateDateTime', 'EXTRA_LASTUPDATEDATETIME'),
    ('NationalID', 'NATIONALIDENTIFIER'),
    ('SourcePatientNumber', 'PATIENTNUMB

Création de la query pour FileTypeMapping


In [20]:
# Création de la requête d'insertion pour FileTypeMapping
query_mapping = ""

for index, row in data.iterrows():
    db_field = row['Nifi Input file']
    mapped_field = row['Nifi Output file']
    file_type = row['TypeFichier']
    query_mapping += f"INSERT INTO FileTypeMapping (FileType, FieldMappingID)\nSELECT '{file_type}', ID FROM FieldMapping WHERE FileInputField = '{db_field}' AND FieldOutputField = '{mapped_field}';\n\n"

# Affichage de la requête
print(query_mapping)


INSERT INTO FileTypeMapping (FileType, FieldMappingID)
SELECT 'Patient ', ID FROM FieldMapping WHERE FileInputField = 'MRN Number' AND FieldOutputField = 'PATIENTNUMBER';

INSERT INTO FileTypeMapping (FileType, FieldMappingID)
SELECT 'Patient ', ID FROM FieldMapping WHERE FileInputField = 'Hospital' AND FieldOutputField = 'EXTRA_HOSPITAL';

INSERT INTO FileTypeMapping (FileType, FieldMappingID)
SELECT 'Patient ', ID FROM FieldMapping WHERE FileInputField = 'DateOfBirth' AND FieldOutputField = 'DATEOFBIRTH';

INSERT INTO FileTypeMapping (FileType, FieldMappingID)
SELECT 'Patient ', ID FROM FieldMapping WHERE FileInputField = 'Gender' AND FieldOutputField = 'GENDER';

INSERT INTO FileTypeMapping (FileType, FieldMappingID)
SELECT 'Patient ', ID FROM FieldMapping WHERE FileInputField = 'PatientDeceased' AND FieldOutputField = 'EXTRA_PATIENTDECEASED';

INSERT INTO FileTypeMapping (FileType, FieldMappingID)
SELECT 'Patient ', ID FROM FieldMapping WHERE FileInputField = 'DateofDeath' AND Fiel